In this lab we will examining how to chain outputs from one prompt to the next.

In [167]:
from operator import itemgetter

from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.llms.bedrock import Bedrock
import boto3

REGION_NAME="us-east-1" ## change to your region
PROFILE_NAME="lza-comm-gss"  ## change to your desired aws credential profile
## ensure Anthropic Claude is enabled in your AWS Account.
named_profile = boto3.session.Session(profile_name=PROFILE_NAME)
bedrock_client = named_profile.client('bedrock-runtime')
print('Initalizing Anthropic Claude v2')
model = Bedrock(
    client=bedrock_client,
    model_id="anthropic.claude-v2",
    endpoint_url="https://bedrock-runtime." + REGION_NAME + ".amazonaws.com",
    model_kwargs={"temperature": 0}
)

Initalizing Anthropic Claude v2


LLMs often need specific instructions and structure in order to reason through a task.  The more complex the task, the more structure it needs. 

### Chains
In this task we will be asking Bedrock - given a type of food, what country is famous for that food? what is the capital of that country (which requires the output from first question) and what is a recipe for that food (recalling the original input)?

Notice the two separate prompts one to answer the first part - the country, which will pass down to the second prompt. 

[We use LCEL to construct the chain1 for the first prompt.](https://python.langchain.com/docs/expression_language/get_started)

In the second chain, notice the second prompt has two inputs - country and food - so we must pass them in a dictionary - country must call the output of the first chain, and then we must use the item getter to get the original input.

### Model differences
Notice we must define a model for each chain - it's important to know that different models are better at different tasks - like text generation vs image generation.  We will explore this in a different lab.  In this lab, we will use Anthropic for both.

We then invoke chain 2 (which calls chain1), and pass it a dictionary with food as the key and whatever food we want to test this with as the value.

In [168]:


prompt1 = ChatPromptTemplate.from_template("what country is famous for {food} ?")
prompt2 = ChatPromptTemplate.from_template(
    "what the capital city of {country} in? provide a recipe for {food}"
)
chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"country": chain1, "food": itemgetter("food")}
    | prompt2
    | model
    | StrOutputParser()
)

print(chain2.invoke({"food": "pizza"}))

ValueError: Error raised by bedrock service: The SSO session associated with this profile has expired or is otherwise invalid. To refresh this SSO session run aws sso login with the corresponding profile.

Let's do something a little more complex.

In [180]:
from langchain_core.runnables import RunnablePassthrough

prompt1 = ChatPromptTemplate.from_template(
    "generate a {attribute} color. Return the name of the color and nothing else."
)
prompt2 = ChatPromptTemplate.from_template(
    "what is a fruit of color: {color}. Return the name of the fruit and nothing else."
)
prompt3 = ChatPromptTemplate.from_template(
    "what is a country with a flag that has the color: {color}. Return the name of the country and nothing else:"
)

prompt4 = ChatPromptTemplate.from_template(
    "What is the color of {fruit} and the flag of {country}?"
)

model_parser = model | StrOutputParser()
color_to_fruit = prompt2 | model_parser
color_to_country = prompt3 | model_parser
color_generator = (
    {"attribute": RunnablePassthrough()} | prompt1 | {"color": model_parser}
)

question_generator = (
   color_generator| {"fruit": color_to_fruit, "country": color_to_country} | prompt4
)
prompt = question_generator.invoke("warm")
model.invoke(prompt)

' Orange is a reddish-yellow color. The flag of the Netherlands consists of three horizontal bands of red, white, and blue.'

## Break this down. 

There's a lot going on here so let's think about logically what the LLM is doing.

The final question we want to ask is: given a fruit and a country - what is the color of the fruit and the color of the flag of that country?

Looking at the input we pass the model - we give an a attribute "warm"... so how do does it go from a random attribue to fruit and country?

## Prompt #1

We see that the first prompt - takes an attribute and generates a color with that attribute.



```python
prompt1 = ChatPromptTemplate.from_template(
    "generate a {attribute} color. Return the name of the color and nothing else. ."
)
```

Try it below.  

In [ ]:
color_generator = (
    {"attribute": RunnablePassthrough()} | prompt1 | {"color": model_parser}
)
color_generator.invoke({"attribute": "warm"})


It takes the attribute warm and it returns orange.  Try it with different adjectives.

Note that the input is passed as a RunnablePassthrough - this essentially just means - return the  initial argument that is passed to this sequence

## Sequences

If we print out the type of color_to_country object - it is a runnable sequence, where the output of one object is the input of the next.  Color_generator and color_to_fruit are both runnable sequences which is why we can pipe to one another. 

```
<class 'langchain_core.runnables.base.RunnableSequence'>
```
Printing the value we get - 
```
first=ChatPromptTemplate(input_variables=['color'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['color'], template='what is a country with a flag that has the color: {color}. Return the name of the country and nothing else:'))]) last=Bedrock(client=<botocore.client.BedrockRuntime object at 0x120554f10>, model_id='anthropic.claude-v2', model_kwargs={'temperature': 0}, endpoint_url='https://bedrock-runtime.us-east-1.amazonaws.com')
| StrOutputParser()
```

If we invoke - this with sequence individually - we get a response to what country has a flag of color the given color?

In [ ]:
print(color_to_country.invoke({"color": "red"}))

## Prompt #2

The second prompt takes a color and returns a fruit of that color.  Which is SUPER convenient since the output of the first prompt is a color!

```python
prompt2 = ChatPromptTemplate.from_template(
    "what is a fruit of color: {color}. Return the name of the fruit and nothing else."
)
```

So for prompt 2 to perform, it takes the attribute input pass it to the first prompt, and then take that output and pass it to the second prompt.

In [ ]:

fruit_generator= color_generator | {"color": color_to_fruit} | prompt2|{"fruit": model_parser}

fruit_generator.invoke({"attribute": "warm"})
# model.invoke(prompt)



## Prompt #3

Prompt # 3 takes a color and returns a country with a flag that has a flag of that color.

```
prompt3 = ChatPromptTemplate.from_template(
    "what is a country with a flag that has the color: {color}. Return the name of the country and nothing else:"
)
```

We use similar logic as the previous step

In [ ]:


country_generator= color_generator | {"color": color_to_country} | prompt3|{"country": model_parser}

country_generator.invoke({"attribute": "warm"})


Your answer may be a little silly - for example, when testing this, I used "warm" and it returned Orange in the Prompt #1 output and netherlands for Prompt #3.  Netherlands does not have an orange flag... but let's pretend this is correct for now.  This is a good example of issues that can override on over-reliance of LLMs.

Let's string it all together.  

We want to construct our final prompt to which we will actually pass our LLM - 

Prompt #2 and Prompt #3 rely on color_generator so we use the result of that as the input for the chain.  In the next chain we define a dictionary - 

```json
{"fruit": color_to_fruit,
 "country": color_to_country }
```

notice the keys have same name as the attributes used in prompt 4. 

and the values are the sequences we defined to render the values using our LLM and our prompts.

## Finally

Running the below cell will render the final question that we will forward to our LLM.

In [ ]:
question_generator = (
   color_generator| {"fruit": color_to_fruit, "country": color_to_country } | prompt4
)

prompt = question_generator.invoke("warm")
print(prompt)



And invoke..

In [ ]:
model.invoke(prompt)

Notice this give us some understanding as to why the LLM said Netherlands has an orange flag - it is because it thinks the fruit orange is reddish-yellow.  It uses red to determine the flag - which has red in it.  A situation like this would require more prompt engineering to rectify - but that is for a different time.

## One step further..

Let's add some more complexity.  This example was taken directly from the Langchain documentation - and I just adding some unit testing for each step along the way.  

Let's explore what adding another intermediate question to the chain and changing the final output would look like.

In [183]:
from langchain_core.runnables import RunnablePassthrough

prompt1 = ChatPromptTemplate.from_template(
    "generate a {attribute} color. Return the name of the color and nothing else. ."
)
prompt2 = ChatPromptTemplate.from_template(
    "what is a fruit of color: {color}. Return the name of the fruit and nothing else."
)
prompt3 = ChatPromptTemplate.from_template(
    "what is a country with a flag that has the color: {color}. Return the name of the country and nothing else:"
)
prompt4 = ChatPromptTemplate.from_template(
    "Name a person from {country}? Return the name of the person and nothing else:"
)
prompt5 = ChatPromptTemplate.from_template(
    "What is the name of the {person} and what are the famous for?"
)

model_parser = model | StrOutputParser()
color_to_fruit = prompt2 | model_parser
color_to_country = prompt3 | model_parser
country_to_person = prompt4 | model_parser
color_generator = (
    {"attribute": RunnablePassthrough()} | prompt1 | {"color": model_parser}
)

question_generator = (
   color_generator| 
   {"fruit": color_to_fruit, "country": color_to_country} |
   prompt4 #| 
#   {"person": country_to_person} | 
#   prompt5
)
prompt = question_generator.invoke("warm")
model.invoke(prompt)

' Vincent van Gogh'

In [ ]:
planner = (
    ChatPromptTemplate.from_template("Generate an argument about: {input}")
    | model
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()}
)

arguments_for = (
    ChatPromptTemplate.from_template(
        "List the pros or positive aspects of {base_response}"
    )
    | model
    | StrOutputParser()
)
arguments_against = (
    ChatPromptTemplate.from_template(
        "List the cons or negative aspects of {base_response}"
    )
    | model
    | StrOutputParser()
)

final_responder = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{original_response}"),
            ("human", "Pros:\n{results_1}\n\nCons:\n{results_2}"),
            ("system", "Generate a final response given the critique"),
        ]
    )
    | model
    | StrOutputParser()
)

chain = (
    planner
    | {
        "results_1": arguments_for,
        "results_2": arguments_against,
        "original_response": itemgetter("base_response"),
    }
    | final_responder
)